In [5]:
using PyPlot
using PyCall
@pyimport matplotlib.animation as anim

In [6]:
function record_snapshot(A, ims)
    """Saves figure snapshot for the final animation"""
    im = imshow(copy(A), interpolation="nearest",
    extent=(0.5,size(A,1)+0.5,size(A,2)+0.5,0.5))
    clim(-1,1)
    push!(ims, PyCall.PyObject[im])
end    

function house(x)
    """Computes the Householder transformation for input vector x"""
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)
    v[1] = 1
    if sigma == 0 && x[1] >= 0
        beta = 0
    elseif sigma == 0 && x[1] < 0
        beta = -2
    else
        mu = sqrt(x[1]*x[1] + sigma)
        if x[1] <= 0
            v[1] = x[1] - mu
        else
            v[1] = -sigma / (x[1] + mu)
        end            

        beta = 2 * v[1] * v[1] / (sigma + v[1]*v[1])
        v /= v[1]
    end

    return (v, beta)
end 

house (generic function with 1 method)

In [7]:
# Initialize matrix
n = 128
R = zeros(n,n)
for i=1:n
    R[i,i] = 1
    for j=i+1:n
        R[i,j] = cos( (pi*i*j) / (4*n) )
    end
end
Q = rand(n,n); Q, = qr(Q)
A = Q*R

ffmpeg_installed = true

if ffmpeg_installed && ! isfile("QR.mp4")    
    fig = figure()
    ims = []
    record_snapshot(A, ims)

    # Apply sequence of Householder transformations
    for k=1:n-2
        v, beta = house(A[k:end,k])
        A[k:end,k:end] = A[k:end,k:end] - beta * v * (v' * A[k:end,k:end])
        record_snapshot(A, ims) # This is used to make the movie at the end
    end            

    colorbar()
    ani = anim.ArtistAnimation(fig, ims, interval=100, blit=true)
    ani[:save]("QR.mp4", extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])
end

In [8]:
# call function to display the animation
if isfile("QR.mp4") 
    display("text/html", string("""<video autoplay controls><source src="data:video/x-m4v;base64,""",
        base64encode(open(readbytes,"QR.mp4")),"""" type="video/mp4"></video>"""));
end